In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

TXT_PATH = 'patients_list.txt'

c:\Users\giuli\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
adata = sc.read_10x_mtx(
        '../GSE239626/MTX',
        var_names='gene_symbols',
        gex_only=True,
        cache=True)

In [3]:
adata.obs['dataset'] = 'GSE239626'
adata.obs['disease'] = 'MS'
adata.obs['cell_type'] = 'PBMC'

In [4]:
adata

AnnData object with n_obs × n_vars = 72317 × 36601
    obs: 'dataset', 'disease', 'cell_type'
    var: 'gene_ids', 'feature_types'

In [5]:
obs = adata.obs.index
obs = [ob.split('-')[-1] for ob in obs]
obs = np.unique(obs)
obs

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '3', '4', '5', '6', '7', '8', '9'], dtype='<U2')

In [35]:
patients_info = pd.read_csv(TXT_PATH, sep=',', header=None)
# add columns name
patients_info.columns = ['patiens', 'p', 'placebo/vit', 'patiens_id', 'time', 'prov']
patients_info.head()

,patiens,p,placebo/vit,patiens_id,time,prov
0,GSM7669046,Patient,Placebo,N1,J0,GEX
1,GSM7669047,Patient,Placebo,N1,J0,ADT
2,GSM7669048,Patient,Placebo,N1,M3,GEX
3,GSM7669049,Patient,Placebo,N1,M3,ADT
4,GSM7669050,Patient,Placebo,N2,J0,GEX


In [53]:
def add_info(row):
    number = row.name.split('-')[-1]
    number_id = (int(number) - 1) * 2
    row['patiens'] = patients_info['patiens'][number_id]
    row['patiens_id'] = patients_info['patiens_id'][number_id]
    row['time'] = patients_info['time'][number_id]

    return row

In [54]:
adata.obs = adata.obs.apply(add_info, axis=1)

In [55]:
adata

AnnData object with n_obs × n_vars = 72317 × 36601
    obs: 'dataset', 'disease', 'cell_type', 'patiens', 'patiens_id', 'time'
    var: 'gene_ids', 'feature_types'

In [56]:
adata.obs

,dataset,disease,cell_type,patiens,patiens_id,time
AAACCCAAGACTGTTC-1,GSE239626,MS,PBMC,GSM7669046,N1,J0
AAACCCAAGGATCATA-1,GSE239626,MS,PBMC,GSM7669046,N1,J0
AAACCCAGTTATTCTC-1,GSE239626,MS,PBMC,GSM7669046,N1,J0
AAACCCATCATGAGGG-1,GSE239626,MS,PBMC,GSM7669046,N1,J0
AAACGAAAGCCAGTAG-1,GSE239626,MS,PBMC,GSM7669046,N1,J0
...,...,...,...,...,...,...
TTTGTTGAGCGTCGAA-20,GSE239626,MS,PBMC,GSM7669084,N5,M3
TTTGTTGAGTGTACCT-20,GSE239626,MS,PBMC,GSM7669084,N5,M3
TTTGTTGCAATTGCAC-20,GSE239626,MS,PBMC,GSM7669084,N5,M3
TTTGTTGCATTCAGCA-20,GSE239626,MS,PBMC,GSM7669084,N5,M3


In [57]:
adata.write('GSE239626.h5ad')